In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd #making some of our usual imports inorder to solve the problem in hand 
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [ ]:
from tensorflow.keras.models import Sequential #importing our deep learing libraries
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_dir="../input/dogs-cats-images/dog vs cat/dataset/test_set" #getting our testing data
train_dir="../input/dogs-cats-images/dog vs cat/dataset/training_set" #getting our training data

train_dir_cats = train_dir + '/cats' #storing the cats training images
train_dir_dogs = train_dir + '/dogs' #storing the dogs training images
test_dir_cats = test_dir + '/cats' #storing the cats testing images
test_dir_dogs = test_dir + '/dogs' #storing the dogs testing images

In [ ]:
print('number of cats training images - ',len(os.listdir(train_dir_cats))) #printing the number of cat training images
print('number of dogs training images - ',len(os.listdir(train_dir_dogs))) #printing the number of dog training images
print('number of cats testing images - ',len(os.listdir(test_dir_cats))) #printing the number of cat testing immages
print('number of dogs testing images - ',len(os.listdir(test_dir_dogs))) #printing the number of dog testing images

In [ ]:
data_generator = ImageDataGenerator(rescale = 1./250,zoom_range = 0.2) #we are converting our RGB photo into array numbers for better computation and processing of our model

In [ ]:
batch_size = 8 #accessing all our data both training and testing
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                  target_size = (150,150),
                                                  batch_size = batch_size,
                                                  class_mode  = 'binary')
testing_data = data_generator.flow_from_directory(directory = test_dir,
                                                  target_size = (150,150),
                                                  batch_size = batch_size,
                                                  class_mode  = 'binary')

In [ ]:
model = Sequential() #making our CNN
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = training_data.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary() #to check how how our model appears to be

In [ ]:
fitted_model = model.fit_generator(training_data,
                        steps_per_epoch = 1000,
                        epochs = 25,
                        validation_data = testing_data,
                        validation_steps = 1000)

In [ ]:
def testing_image(image_directory): #testing out our model
    test_image = image.load_img(image_directory, target_size = (150, 150))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][0]  == 1:
        prediction = 'Dog'
    else:
        prediction = 'Cat'
    return prediction

In [ ]:
print(testing_image(test_dir + '/cats/cat.4003.jpg')) #making prediction

Our model is 79% accurate on our testi**ng set and almost 85% accurate on Training set!